# BMI/CS 576 Fall 2022 - HW4
The objectives of this homework are to practice with

* Tree-based dynamic programming
* Using an HMM for prediction with Viterbi
* Tree space search and parsimony
* Markov chains

## HW policies
Before starting this homework, please read over the [homework policies](https://canvas.wisc.edu/courses/321084/pages/hw-policies) for this course.  In particular, note that homeworks are to be completed *individually* and plagiarism from any source (with the one exception noted below) will be considered **academic misconduct**.

You are welcome to use any code from the weekly notebooks (including the official solutions) in your solutions to the HW.

## Modules for this HW

In [ ]:
import toytree
import fasta
import submatrix

## PROBLEM 1: Tree likelihood (40 points)

Given character states at the leaves of a tree and weights for character substitutions, the weighted parsimony algorithm computes the minimum cost assignment of character states to the ancestral nodes of the tree.  The motivation for this task is that it provides a score for a tree that can be used compare how well the tree fits the data (the characters states at the leaves) relative to other possible tree structures.  An alternative to the weighted parsimony score of a tree is the *likelihood* of the character states at the leaves given the tree and a probabilistic model of how character states evolve along a tree.  In this problem we will implement a dynamic programming algorithm for computing the tree likelihood, which is similar in form to the weighted parsimony algorithm.

In this problem, the generative probabilistic model that we will assume is a simple one in which:
* The character state at the root node is drawn from a categorical distribution $q$, i.e., the prior probability that the character at the root node is $a$ is $q(a)$.
* Character states are generated from the root down to the leaves.  At an internal node of the tree that has a parent node with character state $a$, the probability that the node takes character state $b$ is $S(a,b)$.

The *likelihood* is the marginal probability of the character states at the leaves, given the tree.  It can be efficiently calculated with the following dynamic programming algorithm.  Let $R_i(a)$ be the likelihood of the character states of leaf nodes of the subtree rooted by node $i$ given that node $i$ has the character state $a$. 

**initialization**

For all leaf nodes $i$, set 

$R_i(a) = \left\{\begin{array}{ll}
1 & \textrm{if leaf node $i$ has character state $a$} \\
0 & \textrm{otherwise}
\end{array}\right.$

**recurrence**

In a *postorder* traversal of the tree, for each internal node $i$ with children $j$ and $k$ compute

$R_i(a) = \left(\sum_b R_j(b) S(a, b)\right) \left(\sum_c R_k(c) S(a, c)\right)$

**termination**

Let $root$ be the index of the root node

return $\textrm{likelihood} = \sum_a R_{root}(a) q(a)$

Note that this algorithm is very similar to the weighted parsimony algorithm with minimums replaced by sums and sums replaced by products.  There is no traceback for this algorithm as it computes a sum over all possible assignments to the internal nodes. This algorithm is a variation of what is known as *Felsenstein's algorithm*, which is described in section 8.3 of the textbook.  Implement this algorithm in the function `tree_likelihood` below.  You may find useful the form of the implementation of `fitch_score_and_min_cost_states` in the Day 16 notebook.

Tests for Problem 1 are found at the bottom of this notebook.

### Probabilistic substitution matrices to be used in this assignment

In [ ]:
DNA = "ACGT"
basic_dna_S = submatrix.match_mismatch_matrix(0.85, 0.05, DNA)
basic_dna_q = {a: 0.25 for a in DNA}

print("basic_dna_S = ")
submatrix.print_matrix(basic_dna_S)
print("basic_dna_q = ")
submatrix.print_vector(basic_dna_q)

In [ ]:
def tree_likelihood(tree, leaf_states, S, q, alphabet = DNA):
    """Computes the likelihood of the leaf states given the tree and model parameters (S & q)

    Args:
        tree: a toytree tree.
        leaf_states: a dictionary mapping leaf names to characters.
        S: a substitution matrix (represented as a dictionary with tuples as keys)
           where S[a, b] is the conditional probability of a child node taking
           character state b given that its parent has character state a.
        q: a dictionary with q[a] giving the prior probability of the root node 
           taking character state a.
        alphabet: a string specifying the possible character states that each node may take.
    Returns:
        The marginal probability of the leaf character states.
    """    
    ###
    ### YOUR CODE HERE
    ###


## PROBLEM 2: Inferring recombination within the SARS-CoV-2 spike gene (20 points)

### Recombination
An important phenomenon in the evolution of genomes is that of *recombination*.  In general, recombination is a process in which a new "child" genomic sequence is created from a mixture of two more "parental" genomic sequences.  In humans, recombination regularly occurs during meiosis via a mechanism called "crossing over."  During a crossing over event in humans, the two homologous copies of a chromosome (one from dad and the other from mom) in a cell can exchange homologous parts.  For example, if the two homologous parental chromosomes are

    MMMMMMMMMMMMMMMMMMMMMMMMMMM

and

    DDDDDDDDDDDDDDDDDDDDDDDDDDD

then, for example, after one crossing over event, you can get a new chromosome that looks like

    MMMMMMMMDDDDDDDDDDDDDDDDDDD

or with two crossing over events you can get a chromosome looking like

    MMMMMMMMMMMMMMDDDDDDMMMMMMM

With further crossing over events, perhaps in later generations, chromosomes become complex mosaics of of the original parental chromosomes.  For a fun video illustrating this process in humans (via legos), see this [YouTube video from Science News](https://www.youtube.com/watch?v=RZWB_xt0chY).

Recombination of various forms occurs across the tree of life, including in bacteria and viruses. In this problem, we will explore the possibility of a recombination event in the evolutionary history of SARS-CoV-2.

### Possible recombination within the spike gene in SARS-CoV-2
Recent studies have shown that the pandemic-causing SARS-CoV-2 virus most likely originated from bats ([Wu et al. 2020](http://dx.doi.org/10.1038/s41586-020-2008-3)). However, it still remains unclear whether the virus was transmitted directly to humans from bats, or if there was some other intermediate species. High similarity between SARS-CoV-2 and a coronavirus found in [pangolins](https://en.wikipedia.org/wiki/Pangolin) in the region of the genome containing the spike (S) protein has suggested that pangolins may have played a role (e.g., [Zhang et al. 2020](http://dx.doi.org/10.1016/j.cub.2020.03.022)) and that SARS-CoV-2 may be the result of recombination between bat and pangolin coronaviruses.  Other studies ([Boni et al. 2020](http://dx.doi.org/10.1038/s41564-020-0771-4)) have presented analyses that refute this hypothesis.

In this problem you will analyze an alignment of the spike gene from SARS-CoV-2 and from closely related viruses found in bats and pangolins. Below is code for reading in these sequences.  The sequences in the alignment are spike genes from viruses in human (H), bat (B), pangolin (P), and an outgroup (O) bat.

In [ ]:
alignment_filename = "spike_gene_alignment.fasta"
spike_gene_alignment_recs = fasta.read_sequences_from_fasta_file(alignment_filename)
spike_gene_names, spike_gene_alignment = zip(*spike_gene_alignment_recs)
print(spike_gene_names)

In [ ]:
# In this problem, it will be helpful to have a list of the columns of the alignment
def transpose_alignment(alignment):
    """Returns a column-based alignment from a row-based alignment or vice versa"""
    return list(map(''.join, zip(*alignment)))

spike_gene_alignment_columns = transpose_alignment(spike_gene_alignment)

### PhyloHMMs for detecting recombination from multiple alignments

In this problem, we will identify regions of the spike gene that have potentially undergone recombination using a hidden Markov model (HMM).  Specifically, we will devise a Phylogenetic hidden Markov model (PhyloHMM) in which the the observed sequence is the sequence of *columns* in an *ungapped* multiple alignment and the hidden state at each position is the phylogenetic tree that represents the true evolutionary history of that position.  The model will allow for the true tree to change from position to position, which can happen due to recombination or other mechanisms that transfer pieces of genetic material from one genome to another.  Shown below is an example state transition diagram for a PhyloHMM that generates a multiple alignment of three taxa (A, B, and C).  In this example, there are three states corresponding to the three possible rooted trees of the three taxa.  Shown in dashed boxes are the emission probabilities of each state.  Note that each state emits an ungapped column of a multiple alignment (i.e., a DNA character for each of the three taxa).

![Example PhyloHMM](phylohmm.png)

**(a)** Construct a PhyloHMM model for the spike gene alignment as an instance of the `hmm.HiddenMarkovModel` class.  Assign this instance to the variable `phylohmm`.  Your HMM should have the following characteristics:
* The states should represent all possible rooted trees of four taxa (H, B, P, and O).  *Note: O is not required to be an outgroup in these trees.*
* The emission characters should be all possible columns of an *ungapped* multiple alignment of four DNA sequences.
* The emission probabilities should be calculated by your `tree_likelihood` function from problem 1.  Use the `basic_dna_S` substitution matrix and the `basic_dna_q` prior probabilities.
* The transition probability matrix should be parameterized by a single parameter $r = 0.01$, with the self-transition probabilities set to $1-r$ and the remaining transition probability ($r$) evenly divided across the other states.  In the example PhyloHMM depicted above, this scheme is used with $r = 0.1$.
* The initial probabilities should be a uniform distribution.

*Important note: the `hmm.HiddenMarkovModel` class has been changed slightly from that used in the weekly notebooks to allow for the states and characters to be lists of strings.  This will allow you to use a string instead of a single character to represent your states and observed characters.*

**(b)** Run the Viterbi algorithm with your model and the spike gene alignment. Examine the path of hidden states predicted by Viterbi.  Does this path suggest that there was a recombination event in the evolutionary history of the SARS-CoV-2 spike gene?  Explain your answer.

**(c)** Create another PhyloHMM model that is the same as in (a) but with $r = 0.0001$.  Assign it to the variable `phylohmm2`.  Repeat part (b) with this model.

### 2a (10 points)

In [ ]:
import hmm
###
### Your solution to 2(a)   
###


#### Some hidden automated tests of your phylohmm

In [ ]:
# p2_number_of_states (2 points)
assert isinstance(phylohmm, hmm.HiddenMarkovModel)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# p2_number_of_chars (2 points)
assert isinstance(phylohmm, hmm.HiddenMarkovModel)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# p2_emission_dimension (1 point)
assert isinstance(phylohmm, hmm.HiddenMarkovModel)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# p2_emission_values (1 point)
assert isinstance(phylohmm, hmm.HiddenMarkovModel)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# p2_transition_dimension (1 point)
assert isinstance(phylohmm, hmm.HiddenMarkovModel)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# p2_transition_values (1 point)
assert isinstance(phylohmm, hmm.HiddenMarkovModel)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


### 2b (7 points)

In [ ]:
###
### Your coded solution to 2(b)  
###


###
### Your written solution to 2(b)  
###


### 2c (3 points)

In [ ]:
###
### Your coded solution to 2(C)  
###


###
### Your written solution to 2(c)  
###


## PROBLEM 3: Branch and Bound with unweighted Parsimony (25 points)

Suppose we wish to find an unrooted tree with the minimum unweighted parsimony score for five taxa: 1,2,3,4,and 5, which have character states $T, T, G, G, T$, respectively.  In this problem, we will use the first branch and bound method described in the [Day 16 Tree space search lecture](https://canvas.wisc.edu/courses/321084/pages/day-16-online-lecture-phylogenetic-trees-tree-space-search?module_item_id=5345701) (slide 7: "Exact Method: Branch and Bound") to find such a tree.  We will use the unweighted parsimony score of a partial tree as the lower bound for the score of a full tree that may be built from it.

**(a)** Manually run the branch and bound algorithm on these data starting with the unrooted tree containg taxa 1, 2, and 3.  At the end of each iteration of the algorithm, list the elements (as newick strings) of the queue with their lower bounds. You do *not* need to show your work with respect to computing the parsimony score of each (partial) tree.

**(b)** For how many (partial) trees did you have to compute a parsimony score during the algorithm in part (a)?  How does this compare to the number of possible unrooted trees of five taxa?

###
### Your solution to Problem 3a here
###


###
### Your solution to Problem 3b here
###


## PROBLEM 4: Markov chain parameter estimation and likelihood (15 points)

Suppose we are given the following five DNA sequences. In this problem, we will model these of sequences using a simple Markov chain with a state for each of the four DNA bases.

$\begin{eqnarray}
x_1 & = & \mathrm{\tt CCAA} \\
x_2 & = & \mathrm{\tt ACTT} \\
x_3 & = & \mathrm{\tt GCCG} \\
x_4 & = & \mathrm{\tt CTGC} \\
x_5 & = & \mathrm{\tt TACT} \\
\end{eqnarray}$

**(a)** Using uniform distributions for the transition probabilities and initial state probabilities, calculate the likelihood, $P(x_1, x_2, x_3, x_4, x_5)$, of these sequences (where we assume that each sequence is generated independently from the model).

**(b)** Estimate the parameters (transition and initial probabilities) of the Markov chain using maximum likelihood estimates. Calculate the likelihood of these sequences given these maximum likelihood parameter estimates.

**(c)** Estimate the parameters (transition and initial probabilities) of the Markov chain using Laplace estimates (pseudocount = 1). Calculate the likelihood of these sequences given these Laplace parameter estimates.

###
### Your solution to Problem 4a here
###


###
### Your solution to Problem 4b here
###


###
### Your solution to Problem 4c here
###


## Tests for Problem 1

In [ ]:
# trivial_match (7 points)
x = tree_likelihood(toytree.tree("(A,B);"), {"A": "A", "B": "A"}, basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.18250
print("SUCCESS: trivial_match test case passed!")

In [ ]:
# trivial_mismatch (7 points)
x = tree_likelihood(toytree.tree("(A,B);"), {"A": "A", "B": "C"}, basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.02250
print("SUCCESS: trivial_mismatch test case passed!")

In [ ]:
# triple_match (3 points)
x = tree_likelihood(toytree.tree("(A,(B,C));"), {"A": "T", "B": "T", "C": "T"}, basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.132025
print("SUCCESS: triple_match test case passed!")

In [ ]:
# triple_mismatch (3 points)
x = tree_likelihood(toytree.tree("(A,(B,C));"), {"A": "G", "B": "T", "C": "T"}, basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.016825
print("SUCCESS: triple_mismatch test case passed!")

In [ ]:
# triple_mismatch2 (2 points)
x = tree_likelihood(toytree.tree("(A,(B,C));"), {"A": "T", "B": "G", "C": "T"}, basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.008825
print("SUCCESS: triple_mismatch2 test case passed!")

In [ ]:
# quartet_match (2 points)
x = tree_likelihood(toytree.tree("((A,B),(C,D));"), {"A": "C", "B": "C", "C": "C", "D": "C"}, basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.095514
print("SUCCESS: quartet_match test case passed!")

In [ ]:
# quartet_mismatch (2 points)
x = tree_likelihood(toytree.tree("((A,B),(C,D));"), {"A": "G", "B": "C", "C": "C", "D": "C"}, basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.00641
print("SUCCESS: quartet_mismatch test case passed!")

In [ ]:
# quartet_all_mismatch (2 points)
x = tree_likelihood(toytree.tree("((A,B),(C,D));"), {"A": "G", "B": "C", "C": "T", "D": "A"}, basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.00025
print("SUCCESS: quartet_all_mismatch test case passed!")

In [ ]:
# quartet2_match (1 point)
x = tree_likelihood(toytree.tree("(A,(B,(C,D)));"), {"A": "C", "B": "C", "C": "C", "D": "C"}, basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.095454
print("SUCCESS: quartet2_match test case passed!")

In [ ]:
# quartet2_mismatch (1 point)
x = tree_likelihood(toytree.tree("(A,(B,(C,D)));"), {"A": "G", "B": "C", "C": "C", "D": "C"}, basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.01219
print("SUCCESS: quartet2_mismatch test case passed!")

In [ ]:
# quartet2_mismatch_alt (1 point)
basic_dna_S_alt = submatrix.match_mismatch_matrix(0.91, 0.03, DNA)
basic_dna_q_alt = {"A": 0.2, "C": 0.3, "G": 0.3, "T": 0.2}
x = tree_likelihood(toytree.tree("(A,(B,(C,D)));"), {"A": "G", "B": "C", "C": "C", "D": "C"}, 
                    basic_dna_S_alt, basic_dna_q_alt)
print(round(x, 6))
assert round(x, 6) == 0.011695
print("SUCCESS: quartet2_mismatch_alt test case passed!")

In [ ]:
# large (1 point) (combines quartet2_mismatch and quartet_all_mismatch)
x = tree_likelihood(toytree.tree("((E,(F,(G,H))),((A,B),(C,D)));"),
                    {"A": "G", "B": "C", "C": "T", "D": "A", "E": "G", "F": "C", "G": "C", "H": "C"},
                    basic_dna_S, basic_dna_q)
print(round(x, 6))
assert round(x, 6) == 0.000003
print("SUCCESS: large test case passed!")

### Hidden tests (9 points total)

In [ ]:
# p1_hidden1 (3 points)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# p1_hidden2 (3 points)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
# p1_hidden3 (3 points)
###
### AUTOGRADER TEST - DO NOT REMOVE
###
